In [ ]:
@app.get("/")
def home():
    return {"message": "FastAPI is running"}

def normalize_address(addr: str) -> str:
    """Normalize and clean address for better matching."""
    addr = addr.lower()
    addr = re.sub(r"[^a-z0-9\s]", "", addr)
    addr = addr.replace("street", "st").replace("avenue", "ave").replace("road", "rd")
    addr = addr.replace("boulevard", "blvd").replace("drive", "dr").replace("lane", "ln")
    addr = addr.replace("place", "pl").replace("court", "ct")
    addr = re.sub(r"\s+", " ", addr).strip()
    return addr

def extract_street_name(full_address: str) -> str:
    """Extract just the street portion (e.g., 'Brest Avenue' from '320 Brest Avenue, Kaduna')."""
    if not full_address:
        return ""
    s = full_address.strip()
    s = re.sub(r'^\d+\s*', '', s)  # remove house number
    s = s.split(',')[0]  # remove city part
    return s.strip()

@app.post("/webhook")
async def webhook(req: Request):
    """Dialogflow webhook to check for lead contamination by street name."""
    data = await req.json()
    intent = data["queryResult"]["intent"]["displayName"]
    params = data["queryResult"].get("parameters", {})

    if intent != "Check Home Safety – Get Address":
        return {"fulfillmentText": "This webhook only handles home safety address checks."}

    # Get user-provided text
    user_input = params.get("raw_address") or params.get("geo-city") or data["queryResult"]["queryText"]
    if not user_input:
        return {"fulfillmentText": "Please share your address so I can check for lead contamination."}

    # Normalize user input
    street = extract_street_name(user_input)
    norm_street = normalize_address(street)

    try:
        conn = get_db_connection()
        cursor = conn.cursor(dictionary=True)

        # Pull addresses from the sampled table
        cursor.execute("SELECT Address, Material FROM sampled")
        rows = cursor.fetchall()

        # Normalize database addresses for fuzzy matching
        db_addresses = [normalize_address(row["Address"]) for row in rows]

        # Use fuzzy matching to find the closest match
        best_match = process.extractOne(norm_street, db_addresses, scorer=fuzz.token_sort_ratio)

        if best_match:
            matched_address = rows[best_match[2]]["Address"]
            material = rows[best_match[2]]["Material"]
            score = best_match[1]

            if score < 60:
                msg = f"❓ I couldn’t confidently find '{street}'. Try using a clearer street name."
            else:
                if "lead" in material.lower():
                    msg = (
                        f"⚠️ I found a close match: '{matched_address}'.\n"
                        f"🚨 This area has been reported with {material} pipes. "
                        f"Consider testing your water supply."
                    )
                else:
                    msg = (
                        f"✅ I found a close match: '{matched_address}'.\n"
                        f"This area is recorded as {material}. No lead contamination detected."
                    )
        else:
            msg = f"❌ I couldn’t find '{street}' in the database."

        cursor.close()
        conn.close()
        return {"fulfillmentText": msg}

    except Exception as e:
        return {"fulfillmentText": f"❌ Error while checking address: {str(e)}"}
